In [15]:
import os
import re
import json

from estnltk import EnvelopingBaseSpan
from estnltk import Text, Layer, Annotation, EnvelopingSpan, Span
from estnltk.converters import json_to_text
from estnltk.layer_operations import extract_sections

In [37]:
all_annotations = []

for file in os.listdir(os.getcwd() + "/vallakohtufailid_json/"):
    if file.endswith(".json"):
        with open(os.getcwd() + "/vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                all_annotations.append(dictionary.get('annotations')[0].get('nertag'))
                
print("Total annotations:", len(all_annotations))

print("Annotations by NE type:")

print( str((all_annotations.count("PER") / len(all_annotations)) * 100)[0:5] + "%", all_annotations.count("PER"), "Isik")
print( str((all_annotations.count("LOC_ORG") / len(all_annotations)) * 100)[0:5] + "%", all_annotations.count("LOC_ORG"), "KO_Koht")
print( str((all_annotations.count("LOC") / len(all_annotations)) * 100)[0:5] + "%", all_annotations.count("LOC"), "Koht")
print( str((all_annotations.count("ORG") / len(all_annotations)) * 100)[0:5] + "%", all_annotations.count("ORG"), "Org")
print( str((all_annotations.count("MISC") / len(all_annotations)) * 100)[0:5] + "%", all_annotations.count("MISC"), "Muu, teadmata, ese")

Total annotations: 27540
Annotations by NE type:
83.97% 23126 Isik
9.923% 2733 KO_Koht
3.660% 1008 Koht
1.521% 419 Org
0.922% 254 Muu, teadmata, ese
